flask() 코드 {모델과 웹 프레임워크 연결}

In [ ]:
from flask import Flask , render_template,request,url_for,send_from_directory,session
from werkzeug.utils import secure_filename
import itertools
import imageio
import os
import cv2
import numpy as np
from flask import jsonify
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,GlobalMaxPooling2D,GlobalMaxPooling2D
from keras.applications import VGG16,ResNet50
import keras.utils as image
from imutils import paths



app = Flask(__name__)
app.secret_key="super secret key"
app.config['UPLOAD_FOLDER'] ='C:/Users/dydrk/aii_project/venvv/static/images/'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


@app.route("/predict",methods=['GET','POST'])
def predict():
  if request.method =='POST':
    file=request.files['image']
    if not file: return render_template('index.html', label="No Files") 
    #Path='C:/Users/dydrk/aii_project/venvv/static/images/'+str(file.filename)
    file.save(os.path.join(app.config['UPLOAD_FOLDER'],file.filename)) #저장시킨 파일..
    model=keras.models.load_model('C:/Users/dydrk/aii_project/venvv/my_cofee.h5')  
    Path=(os.path.join(app.config['UPLOAD_FOLDER'],file.filename))
    
    img = image.load_img(Path) #저장시킨 파일 불러오기
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0) 
    images = np.vstack([img])/255
    classes = model.predict(images, batch_size=32)
    clas = list(itertools.chain(*classes))
    print(clas)
    if clas[0]>=max(clas):
      coffee='green'
    elif clas[1]>=max(clas):
      coffee ='light'
    elif clas[2]>=max(clas):
      coffee='medium'
    elif clas[3]>=max(clas):
       coffee='dark'
             
    return render_template('index.html', data=coffee)

   
@app.route('/') #파이썬 데코레이터 문법 =라우팅
def index():
    return render_template("index.html")
    

if __name__ == '__main__':
    #model = keras.models.load_model('my_cofee.h5')
    print(("*Loading Keras model and Flask starting server..."
           "please wait until server has fully started"))
    app.run(debug=True,host="0.0.0.0",port=8001)





#@app.route('/user/<user_name>/<int:user_id>')
#def user(user_name, user_id):
  #  return f'Hello,{user_name}({user_id})!'

네이버 이미지 크롤링

In [ ]:
from selenium import webdriver
import time
from urllib.parse import quote_plus
from selenium.webdriver.common.by import By
from urllib.request import urlopen
import os

def save_images(images,save_path):
  for index,image in enumerate(images[:1000]):
    src=image.get_attribute('src')
    t=urlopen(src).read()
    file=open(os.path.join(save_path,str(index+1)+'.jpg'),"wb")
    file.write(t)
    print("img save"+save_path+str(index+1)+".jpg")

def create_folder_if_not_exists(directory):
  try:
    if not os.path.exists(directory):
      os.makedirs(directory)
    except OSError:
      print('Error: Creating directory.'+directory)

def make_url(search_term):
  base_url='https://search.naver.com/search.naver?where=image&section=image&query='

def crawl_images(search_term):
  url=make_url(search_term)

  browser=webdriver.Chrome('chromedriver')
  browser.implicitly_wait(30)
  browser.get(url)
  time.sleep(5)

  for_in range(10000):
    browser.execute_script("window.scrollBy(0,30000)")
  
  images=browser.find_elements(By.CLASS_NAME,"_image")

  save_path="C:\coffee"+search_term+"/"
  create_folder_if_not_exists(save_path)
  save_images(images,save_path)
  print(search_term+"저장 성공!")
  browser.close()

if__name__=='__main__':
  crawl_images(input('원하는 검색어:'))

구글 이미지 크롤링

In [ ]:
from selenium import webdriver
import time
from urllib.parse import quote_plus
from selenium.webdriver.common.by import By
from urllib.request import urlopen
import os


driver = webdriver.Chrome()
driver.get("https://www.google.co.kr/imghp?h1=ko&tab=ri&ogb1")

elem=driver.find_element(By.NAME,"q")
elem.send_keys("생두") #생두, 라이트 로스팅, 미디엄 로스팅, 다크 로스팅으로 바꿔가며 크롤링
elem.send_keys(Keys.RETURN)
SCROLL_PAUSE_TIME=1
last_height=driver.execute_script("return document.body.scrollHeight")
while True:
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight");)
  time.sleep(SCROLL_PAUSE_TIME)
  new_height=driver.execute_script("return document.body.scrollHeight")
  if new_height==last_height:
    try:
      driver.find_element(By.CSS_SELECTOR,".mye4qd").click()
    except:
       break
    last_height=new_height

images=driver.find_elements(By.CSS_SELECTOR,".rg_i.Q4LuWd")
count=1
for image in images:
  image.click()
  time.sleep(3)
  imgUrl=driver.find_element(By.CSS_SELECTOR,".n3VNCb").get_attribute("src")
  urllib.request.urlretrieve(imgUrl,str(count)+".jpg")
  count=count+1